# 講義用Notebook環境のイメージ登録

---

講義用のNotebook環境として利用するSingle-user Serverのコンテナイメージを登録します。

## 概要

CoursewareHubでは管理ユーザ（講師）が[repo2docker](https://github.com/jupyterhub/repo2docker)を利用して講義用のNotebook実行環境をカスタマイズする機能を備えています。このNotebookでは講義用Notebook環境のコンテナイメージを登録、管理する手順について説明します。

### repo2docker

repo2dockerはレポジトリ(GitHub, GitLab, gitレポジトリ, ...)から取得した構成ファイルをコンテナイメージとしてビルドし、Dockerレジストリに格納するためのツールです。

repo2dockerでは[pip](https://pip.pypa.io/)の`requirements.txt`や[conda](https://conda.io/)の`environment.yml`など、既存のツールの設定ファイルを構成ファイルとしてそのまま利用してコンテナイメージを定義することができます。そのためDockerfileを直接記述することなく、様々な講義用のコンテナイメージを作成することができます。


repo2dockerの構成ファイルとして利用できる主なものを以下に示します。

* environment.yml
  - [conda](https://conda.io/)の構成ファイル
  - Python, R, C/C++ のパッケージを含めた全てのcondaのパッケージを指定することができる
* requirements.txt
  - pipで定められている[書式](https://pip.pypa.io/en/latest/reference/requirements-file-format/)でPythonのパッケージを指定する
  - この書式ではPythonの実行環境を指定することができないので、別ファイル`runtime.txt`でPythonのバージョンを指定する
* apt.txt
  - apt-getでインストールするDebianパッケージのリストを指定する
* runtime.txt
  - ランタイム（PythonやRなど）のバージョンを指定する
  - `environment.txt`, `Project.toml`などランタイムを指定することができる構成ファイルがある場合、このファイルの指定は無視される
* install.R
  - R/RStudioにインストールするRライブラリを指定する
* Project.toml
  - Julia のバージョンとJulia パッケージのリストを指定する

repo2dockerが対応している他の構成ファイルや、記述方法の詳細については[repo2docker - Configuration Files](https://repo2docker.readthedocs.io/en/latest/config_files.html)を参照してください。

GitHubで公開されている[Binder Examples](https://github.com/binder-examples)にはPythonやRなどを利用した様々な構成ファイルの例が示されていて、記述の参考にすることができます。ただし後に説明するようにCoursewareHubの講義用Notebook環境のコンテナイメージとして利用するにはいくつかの条件を満たしている必要があります。Binder Examplesに示されているプロジェクトはこの条件を満たしていないため、そのままCoursewareHubのカスタムイメージ定義として利用することはできません。

### CoursewareHubのイメージ作成機能

CoursewareHubにはrepo2dockerによるコンテナ作成機能のプラグインとして[cwh-repo2docker](https://github.com/NII-cloud-operation/CoursewareHub-LC_platform/blob/master/jupyterhub/cwh-repo2docker/)が組み込まれています。CoursewareHubのイメージ作成機能に関わる部分を次図に示します。

![イメージ作成](images/cw-731-01.png)

イメージのビルド、登録を行うrepo2dockerはdockerコンテナとしてmanagerノードで実行されます。このコンテナはCoursewareHubのイメージ作成機能拡張により起動され、カスタムイメージのビルドとdockerレジストリの登録を行います。イメージを定義する構成ファイルはGitHubなどのレポジトリから取得されます。取得先となるレポジトリに関する情報は（CoursewareHubにより拡張された）JupyterHubの管理画面で指定します。repo2dockerがコンテナイメージのビルドに成功するとmanagerノードで実行しているプライベートdockerレジストリに登録します。

CoursewareHubでは登録されているイメージを管理するためにデータベース等を利用せずにDockerレジストリに登録されている情報のみを用います。管理するための情報はコンテナイメージの[ラベル](https://docs.docker.com/config/labels-custom-metadata/)に付与します。

## カスタムイメージの定義

CoursewareHubで利用する講義用のカスタムイメージについて説明します。

### レポジトリ

イメージを定義するレポジトリ（GitHub, git, ...）にはrepo2dockerの仕様に沿った構成ファイルを配置します。repo2dockerの構成ファイルとして指定できるものには`environment.yml`(conda), `requirements.txt`(pip), `apt.txt`(apt-get)などがあります。詳細については「[repo2docker - Configuration Files](https://repo2docker.readthedocs.io/en/latest/config_files.html)」を参照してください。

レポジトリ内に構成ファイルを配置する場所は「[repo2docker - Where to put configuration files](https://repo2docker.readthedocs.io/en/latest/usage.html#usage-config-file-location)」に記されているように、以下のいずれかの場所とする必要があります。

* レポジトリのルートディレクトリ
* `binder/`
* `.binder/`


### CoursewareHubで利用するための必要条件

カスタムイメージをCoursewareHubで利用できるようにするための必要条件について説明します。

Single-user ServerのコンテナをCoursewareHubで利用するには、NFS サーバにある各ユーザのホームディレクトリをコンテナからアクセスするために以下の条件を満たす必要があります。

* コンテナのユーザIDとユーザ名がmanagerノードのユーザIDとユーザ名に一致していること
* コンテナのユーザ名がJupyterHubのユーザ名に一致していること

これらの条件を満たすコンテナイメージを作成する方法の一つに、条件を満たすような振る舞いを実現する起動スクリプトをコンテナに配置する方法があります。具体的には[Docker Stacks](https://github.com/jupyter/docker-stacks)の起動スクリプト`start.sh`をrepo2dockerの構成ファイル`start`として配置することでほぼ達成できます。条件を満たすためには以下の対応が追加で必要となります。

* `apt.txt`にsudoを追加する
  * `start.sh`スクリプトで`sudo`コマンドを利用するため
* `start`のsecure_pathを変更する
  * `start.sh`スクリプトから`sudo`コマンドで`jupyterhub-singleuser`を起動
できるようにするため

Docker Stacksの`start.sh`を利用して、カスタムイメージ定義のレポジトリを準備するためのスクリプトを用意しています。

* [init_repo.sh](template/init_repo.sh)

このスクリプトを実行すると、repo2dockerの構成ファイル`start`, `apt.txt`を配置したレポジトリを準備します。その後、必要に応じて`environment.yml`, `requirements.txt`などの構成ファイルを追加することで、簡単にイメージ定義を行うレポジトリを作成することができます。実際の実行例を次の節に示しています。

なお`init_repo.sh`はcondaやPythonnのパッケージで構成されているイメージを定義するレポジトリを準備することを想定しています。Juliaを利用するコンテナイメージを定義するには`start`で`CHOWN_EXTRA`に追加のパスを設定するなど追加の対応作業が必要となります。

### 作成手順

[seaborn](http://seaborn.pydata.org/)を利用できるカスタムイメージを定義するレポジトリを作成する例を示します。

> ここで示す手順で作成したものと同じ内容のものが、カスタムイメージの例として<https://github.com/nii-gakunin-cloud/cwh-custom-image-seaborn>にあります。

gitレポジトリを作成するディレクトリを作成して、初期設定スクリプト`init_repo.sh`を実行します。

```console
mkdir -p cwh-seaborn
cd cwh-seaborn
init_repo.sh
```

初期設定スクリプトにより`apt.txt`, `start`が作成され git の初期化が行われた状態になっています。

```console
$ tree -aF -L 1 .
./
├── .git/
├── apt.txt
└── start*

2 directories, 2 files
```

`apt.txt`にはsudoだけが指定されています。

```console
$ cat apt.txt 
sudo
```

ここから講義用のイメージとして必要なパッケージを指定するための構成ファイルを追加していきます。

まずseabornなどのPythonパッケージをインストールするための`requirements.txt`作成します。

```console
cat > requirements.txt <<EOF
contourpy==1.1.1
cycler==0.12.1
fonttools==4.43.1
kiwisolver==1.4.5
matplotlib==3.8.0
numpy==1.26.1
packaging==23.2
pandas==2.1.1
Pillow==10.1.0
pyparsing==3.1.1
python-dateutil==2.8.2
pytz==2023.3.post1
seaborn==0.13.0
six==1.16.0
tzdata==2023.3
EOF
```

コンテナで利用するPythonのバージョンを指定するための`runtime.txt`を作成します。

```console
echo "python-3.11" > runtime.txt
```

ここまでに作成したrepo2dockerの構成ファイルは以下のようになります。

```console
$ tree -aF -L 1 .     
./
├── .git/
├── apt.txt
├── requirements.txt
├── runtime.txt
└── start*

2 directories, 4 files
```

作成したイメージ定義をGitHubなどにpushします(GitHubにはあらかじめ空のプロジェクトが作成済みであることを想定しています)。

```console
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/xxxxx/cwh-seaborn-001.git
git push -u origin main
```

pushしたレポジトリのURLを次章で説明するカスタムイメージ登録手順に指定することでCoursewareHubに講義用イメージを登録することができます。

### カスタムイメージの定義例


CoursewareHubから利用できるカスタムイメージの定義例が用意してあります。

* <https://github.com/nii-gakunin-cloud/cwh-custom-image-seaborn.git>
    * [seaborn](https://github.com/mwaskom/seaborn)を利用できるようにしたカスタムイメージの定義
* <https://github.com/nii-gakunin-cloud/cwh-custom-image-network-command.git>
    * `dig`, `ping`, `ip` などのネットワークコマンドを利用できるようにしたカスタムイメージの定義

これらのレポジトリをフォークして変更することにより、講義に応じたカスタムイメージを準備することができます。

## カスタムイメージの管理

### カスタムイメージを登録する

管理ユーザ（講師）が講義用のカスタムイメージを登録する手順を示します。

Jupyter Hubの管理画面を表示してください。Classic Notebook Interfaceを利用している場合は[Control Panel]ボタン（下図の赤丸）を選択することで管理画面を表示できます。

![Jupyter Notebook 01](images/cw-731-10.png)

管理画面のツールバーに表示されている[Environments]リンクを選択するとイメージ管理画面（下図）が表示されます。

![管理メニュー](images/cw-731-11.png)

講義用のカスタムイメージを登録するにはイメージ管理画面の[Add New]ボタン（上図の赤丸）を選択してください。イメージ作成ダイアログ（下図）が表示されます。イメージ作成ダイアログの[Repository URL]にイメージを定義したレポジトリのURLを入力してください。また必要に応じて[Reference (git commit)], [Name of the environment]の指定を行ってください。

![登録ダイアログ1](images/cw-731-12.png)

プライベートなレポジトリの場合は、レポジトリにアクセスするために必要となる認証情報を[Credentials (optional)]に表示される入力欄に指定してください（下図）。

![登録ダイアログ2](images/cw-731-13.png)

イメージ作成ダイアログの[Create Environment]ボタンを選択するとイメージのビルド、登録処理が開始されます。イメージ管理画面には対応する行が追加され、Statusなどの表示からビルド中であることがわかります（下図）。

![イメージのビルド中](images/cw-731-14.png)

[Logs]ボタン(上図の赤丸）を選択することによりrepo2dockerで実行されているビルド、登録処理のログを確認することができます（下図）。

![ログダイアログ](images/cw-731-15.png)

イメージのビルド、登録処理が成功した後にイメージ管理画面を表示すると[Status]の表示がチェックのアイコンになり、[Image ID]の値が表示されるようになります（下図）。

![イメージ一覧](images/cw-731-16.png)

### サーバ起動時のイメージ選択

Single-userサーバを起動したときにどのイメージが選択されるかについて説明します。

ユーザの権限により、起動時のイメージ選択に関する振る舞いが異なります。

管理ユーザ（講師）がサーバを起動した場合、複数のイメージが登録されているとイメージの選択画面が表示されます（下図）。

![起動イメージ選択画面](images/cw-731-17.png)

ラジオボタンで起動するイメージを選択し[Start]ボタンを押すと、指定したイメージでサーバが起動します。

一方、管理ユーザ以外のユーザ（受講者）がサーバを起動した場合、イメージ選択画面は表示されず、事前に管理ユーザにより「Default Course Image」として設定されたイメージでサーバが起動します。「Default Course Image」はイメージ管理画面にて設定することができます（下図の赤丸部分）。

![イメージ一覧](images/cw-731-18.png)